In [ ]:
!pip install opencv-python-headless numpy


In [ ]:
import cv2
import numpy as np
import base64
from IPython.display import HTML, display, clear_output
from google.colab import output
from matplotlib import pyplot as plt


In [ ]:
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)
eye_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_eye.xml'
)


In [ ]:
def detect_and_show(img_data):
    clear_output(wait=True)   # ✅ removes OLD output immediately

    img_bytes = base64.b64decode(img_data.split(',')[1])
    img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.2, 5)

    status = "NOT SLEEPING"

    for (x, y, w, h) in faces:
        face_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(face_gray, 1.1, 3)

        if len(eyes) == 0:
            status = "SLEEPING"

        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)

    cv2.putText(
        img,
        f"Status: {status}",
        (10, 35),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255) if status == "SLEEPING" else (0, 255, 0),
        2
    )

    plt.figure(figsize=(6,6))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")


In [ ]:
def process_frame(img_data):
    detect_and_show(img_data)

output.register_callback('notebook.processFrame', process_frame)

display(HTML('''
<video id="video" width="400" autoplay playsinline></video><br><br>

<button onclick="startCamera()">📷 Start Camera</button>
<button onclick="detect()">▶ Detect</button>

<script>
let stream = null;

async function startCamera() {
    if (stream) return;
    stream = await navigator.mediaDevices.getUserMedia({video: true});
    const video = document.getElementById("video");
    video.srcObject = stream;
}

function detect() {
    if (!stream) {
        alert("Start camera first");
        return;
    }

    const video = document.getElementById("video");
    const canvas = document.createElement("canvas");
    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;
    canvas.getContext("2d").drawImage(video, 0, 0);

    // 🔥 Python handles output, JS does NOTHING else
    google.colab.kernel.invokeFunction(
        'notebook.processFrame',
        [canvas.toDataURL('image/jpeg')],
        {}
    );
}
</script>
'''))
